# **Resume and CV auto ML Shortlisting**

In [5]:
import spacy
import pickle
import random


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
train_data = pickle.load(open('/content/drive/MyDrive/train_data.pkl', 'rb'))


In [10]:
len(train_data)

200

In [11]:
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [ ]:
for _, annotation in train_data:
  for ent in annotation['entities']:
    print(ent[2])

start with blank spacy model

In [13]:
nlp = spacy.blank('en')


Train with our custom train_data containing resume with their annotations

and update that in our nlp model

In [14]:

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
    
    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(30):
            print("Statring iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
                
            print(losses)

In [15]:
train_model(train_data)


Statring iteration 0
{'ner': 11613.31604754122}
Statring iteration 1
{'ner': 9848.162704075788}
Statring iteration 2
{'ner': 8045.146713959911}
Statring iteration 3
{'ner': 7753.9564107664655}
Statring iteration 4
{'ner': 6978.712976786129}
Statring iteration 5
{'ner': 9830.201889201124}
Statring iteration 6
{'ner': 5281.667956368961}
Statring iteration 7
{'ner': 6468.824763420379}
Statring iteration 8
{'ner': 4942.961672833037}
Statring iteration 9
{'ner': 4907.074120186795}
Statring iteration 10
{'ner': 4615.2886884837935}
Statring iteration 11
{'ner': 5140.3510118519725}
Statring iteration 12
{'ner': 4546.177023835735}
Statring iteration 13
{'ner': 4209.334645818486}
Statring iteration 14
{'ner': 4109.077401977333}
Statring iteration 15
{'ner': 4742.282748629759}
Statring iteration 16
{'ner': 4281.842551840907}
Statring iteration 17
{'ner': 3067.491373996399}
Statring iteration 18
{'ner': 3865.809773115568}
Statring iteration 19
{'ner': 2852.6774675732895}
Statring iteration 20
{'ne

save model to google drive

In [17]:
nlp.to_disk('/content/drive/MyDrive/NLP NER model/nlp_model')


load the model using spacy library

In [18]:
loaded_nlp_model = spacy.load('/content/drive/MyDrive/NLP NER model/nlp_model')


In [19]:
train_data[0][0]


'Saurabh Saurabh Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Saurabh- Saurabh/87e6b26903460061  Willing to relocate: Anywhere  WORK EXPERIENCE  Developer Support Engineer  Microsoft iGTSC -  Bangalore Urban, Karnataka -  August 2007 to Present  EDUCATION  Certificate of Achievement  Microsoft Virtual Academy  December 2016  SKILLS  C, C++, Data Structure, Java (1 year)  ADDITIONAL INFORMATION  ➢ Managerial Skills: Organizer and volunteer at many inter and intra college events, symposia etc. in college. ➢ Co-curricular: - 1. Selected by College (TISL) to train the first-year engineering students under the Finishing School Program (January 2017) 2. District level soccer and cricket player. 3. Honored at State Level Singing competition; Performed in college.  ➢ Leadership Skills: Effective leadership; associated with many student led organizations; Student leader for Oxygen, A movement for and by Students (2007 &amp; 2009)  DECLARATION  The abovementioned is true to the best o

performing testing on our earlier seen data

In [21]:
doc = loaded_nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Saurabh Saurabh
LOCATION                      - Bengaluru
EMAIL ADDRESS                 - indeed.com/r/Saurabh- Saurabh/87e6b26903460061
DESIGNATION                   - Developer Support Engineer
DEGREE                        - Certificate of Achievement
GRADUATION YEAR               - 2016
SKILLS                        - C, C++, Data Structure, Java (1 year)  ADDITIONAL INFORMATION  ➢ Managerial Skills: Organizer and volunteer at many inter and intra college events, symposia etc. in college. ➢ Co-curricular: - 1. Selected by College (TISL) to train the first-year engineering students under the Finishing School Program (January 2017) 2. District level soccer and cricket player. 3. Honored at State Level Singing competition; Performed in college.  ➢ Leadership Skills: Effective leadership; associated with many student led organizations; Student leader for Oxygen, A movement for and by Students (2007 &amp; 2009)  DECLARATION  The abovementioned is true to 

Testing with unseen data

In [23]:
!pip install PyMuPDF

     |████████████████████████████████| 6.3MB 12.6MB/s 


In [26]:
import sys, fitz
fname = '/content/drive/MyDrive/Smith Resume.pdf'
doc = fitz.open(fname)
doc

Document('/content/drive/MyDrive/Smith Resume.pdf')

In [27]:

text = ""
for page in doc:
    text = text + str(page.getText())

tx = " ".join(text.split('\n'))
print(tx)

Michael Smith  BI / Big Data/ Azure  Manchester, UK- Email me on Indeed: indeed.com/r/falicent/140749dace5dc26f    10+ years of Experience in Designing, Development, Administration, Analysis,  Management  inthe  Business  Intelligence  Data  warehousing,  Client  Server  Technologies, Web-based Applications, cloud solutions and Databases.  Data warehouse: Data analysis, star/ snow flake schema data modeling and design  specific todata warehousing and business intelligence environment.  Database: Experience in database designing, scalability, back-up and recovery,  writing andoptimizing SQL code and Stored Procedures, creating functions, views,  triggers and indexes.   Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL  Azure, StreamAnalytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure  data lake analytics(U-SQL).  Big Data: Worked Azure data lake store/analytics for big data processing and Azure  data factoryto schedule U-SQL jobs. Designed and d

# **RESULT**

In [28]:
doc = loaded_nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Michael Smith
LOCATION                      - Manchester
EMAIL ADDRESS                 - indeed.com/r/falicent/140749dace5dc26f
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COLLEGE NAME                  - The University of Manchester - UK
SKILLS                        - problem solving (Less than 1 year), project lifecycle (Less than 1 year), project  manager (Less than 1 year), t